In [7]:
from pyspark.sql import SparkSession

1. Creamos una SparSession

In [8]:
spark = SparkSession\
    .builder\
        .master("local[*]")\
            .appName('Test')\
                .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/03/01 17:45:03 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/03/01 17:45:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/01 17:45:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/01 17:45:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


1.1 Leemos un archivo .CSV para crear un __DataFrame__

In [3]:
PATH = "/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv"
df = spark.read\
    .option("header", "True") \
        .csv(PATH)

1.2 Aplicamos una __Accion__sobre el __DataFrame__ para mostrar su contenido

In [4]:
df.show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



2. Trabajamos con Esquemas

2.1 Mostramos el esquema de DF anterior __Vemos que todo es string__ y no es lo que queremos.

In [5]:
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: string (nullable = true)
 |-- email: string (nullable = true)



2.2 Inferimos el Esquema usando __".option("inferSchema", True)__

In [153]:
df = spark.read.option("Header","True").option("inferSchema", True).csv(PATH)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: integer (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



2.3 Especificar esquema de forma manual.

In [154]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType 

schema_a = StructType ([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("email", StringType(), True),
])

df = spark.read.schema(schema_a).option("header",True).csv(PATH)
df.show(5)
print(df.printSchema())

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)

None


3.1 Usar **Kwargs en Spark

Se usa para no tener tantos __.option()__ anidados remplazandolo por un unico __.options()__

In [26]:
df = spark.read.options(inferSchema=True, Header=True, delimiter=",").csv(PATH)
df.show(2)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows



4. Spark DF from RDDs

Primero creamos un RDD y Luego lo pasamos a DF

In [27]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('Test_rdd').setMaster("local[*]")
spark_rdd = SparkContext.getOrCreate(conf=conf)

In [53]:
PATH = "/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv"

rdd = spark_rdd.textFile(PATH)
header = rdd.first()
rdd_data = rdd.filter(lambda x: x != header)
#print(rdd_data.count())

In [58]:
rdd_data_split = rdd_data.map(lambda x:x.split(","))
rdd_data_f = rdd_data_split.map(lambda x: [int(x[0]), x[1], x[2], x[3], x[4], int(x[5]), x[6]])
rdd_data_f.first()

[28,
 'Female',
 'Hubert Oliveras',
 'DB',
 '02984',
 59,
 'Annika Hoffman_Naoma Fritts@OOP.com']

4.1 Convertimos el __Rdd__ en un __DataFrame__ usando el Header que sacamos el __Rdd__

In [62]:
lista_headers = header.split(",")
df = rdd_data_f.toDF(lista_headers)

In [63]:
df.show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



In [64]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: long (nullable = true)
 |-- email: string (nullable = true)



4.2 Como el esquema no se puede inferir, lo que hacemos es crear nuestro propio esquema, pero en lugar de usar la __transformacion rdd.toDF()__ usamos __spark.createDataFrame(rdd, schema=)__

In [65]:

schema_rdd = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("email", StringType(), True)
])

mi_df = spark.createDataFrame(rdd_data_f, schema=schema_rdd)
print(mi_df.printSchema())

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)

None


5. Seleccion de columnas

5.1 Notación directa, especificando la columna

In [67]:
mi_df.select("age", "gender").show(2)

+---+------+
|age|gender|
+---+------+
| 28|Female|
| 29|Female|
+---+------+
only showing top 2 rows



5.2 Usando __dot notation__

In [68]:
df.select(df.age, df.email).show(2)

+---+--------------------+
|age|               email|
+---+--------------------+
| 28|Annika Hoffman_Na...|
| 29|Margene Moores_Ma...|
+---+--------------------+
only showing top 2 rows



5.3 Usando __col()__

Es una funcion dentro de la libreria __pyspark.sql.functions__

In [70]:
from pyspark.sql.functions import col

df.select(col("age"), col("gender")).show(5)

+---+------+
|age|gender|
+---+------+
| 28|Female|
| 29|Female|
| 28|  Male|
| 29|Female|
| 28|  Male|
+---+------+
only showing top 5 rows



5.4 Mostrar todas las columnas usando __*__

In [72]:
df.select("*").show(4)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 4 rows



5.5 Select de las columnas usando Index

In [76]:
df.select(df.columns[3:]).show(4)

+------+-----+-----+--------------------+
|course| roll|marks|               email|
+------+-----+-----+--------------------+
|    DB|02984|   59|Annika Hoffman_Na...|
| Cloud|12899|   62|Margene Moores_Ma...|
|    PF|21267|   45|Jeannetta Golden_...|
|    DB|32877|   29|Billi Clore_Mitzi...|
+------+-----+-----+--------------------+
only showing top 4 rows



5.6 Podemos combinar todo lo anterior

In [87]:
df.select(col("age"), df.columns[-1]).show(4)

+---+--------------------+
|age|               email|
+---+--------------------+
| 28|Annika Hoffman_Na...|
| 29|Margene Moores_Ma...|
| 28|Jeannetta Golden_...|
| 29|Billi Clore_Mitzi...|
+---+--------------------+
only showing top 4 rows



6. WithColumn()

6.1 Ejemplo de Casteo de una columna

In [95]:
df.withColumn("roll2", col("roll").cast("Integer")).show(5)

+---+------+----------------+------+-----+-----+--------------------+-----+
|age|gender|            name|course| roll|marks|               email|roll2|
+---+------+----------------+------+-----+-----+--------------------+-----+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...| 2984|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|12899|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|21267|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|32877|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|41487|
+---+------+----------------+------+-----+-----+--------------------+-----+
only showing top 5 rows



6.2 Ejemplo de manipulacion de valores

In [98]:
from pyspark.sql.functions import col

df.withColumn("marks2", col("marks") + 5).show(5)

+---+------+----------------+------+-----+-----+--------------------+------+
|age|gender|            name|course| roll|marks|               email|marks2|
+---+------+----------------+------+-----+-----+--------------------+------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|    64|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|    67|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|    50|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|    34|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|    46|
+---+------+----------------+------+-----+-----+--------------------+------+
only showing top 5 rows



6.3 Crear un valor constante

Con .withColumn se puede crear un valor constante, pero hay que __especificarlo con la funcion lit de .sql.functions__

In [100]:
from pyspark.sql.functions import col, lit

df.withColumn('pais',lit('ARG')).show(5)

+---+------+----------------+------+-----+-----+--------------------+----+
|age|gender|            name|course| roll|marks|               email|pais|
+---+------+----------------+------+-----+-----+--------------------+----+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...| ARG|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...| ARG|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...| ARG|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...| ARG|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...| ARG|
+---+------+----------------+------+-----+-----+--------------------+----+
only showing top 5 rows



6.4 Concatenacion de .withColumn

In [103]:
from pyspark.sql.functions import col

df.withColumn('marks', col('marks') - 10).withColumn('marks', col('marks') + 20).show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   69|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   72|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   55|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   39|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   51|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



7. withColumnRename

Lo usamos para cambiar el nombre de una columna y generar un nuevo __DF__

In [11]:
df_rename = df.withColumnRenamed("age", "edad")
df_rename.show(4)

+----+------+----------------+------+-----+-----+--------------------+
|edad|gender|            name|course| roll|marks|               email|
+----+------+----------------+------+-----+-----+--------------------+
|  28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
|  29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
|  28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
|  29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+----+------+----------------+------+-----+-----+--------------------+
only showing top 4 rows



7.1 Si queremos cambiar el nombre de una variable en tiempo de ejecución usamos __.alias()__

In [17]:
from pyspark.sql.functions import col

df_rename.select(col("edad").alias("Age")).show(5)

+---+
|Age|
+---+
| 28|
| 29|
| 28|
| 29|
| 28|
+---+
only showing top 5 rows



8. Filter en __DataFrames__

In [18]:
from pyspark.sql.functions import col

df.filter(col("course")=='DB').show(5)
df.filter(df.course == "DB").show(5)

+---+------+---------------+------+------+-----+--------------------+
|age|gender|           name|course|  roll|marks|               email|
+---+------+---------------+------+------+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 29|  Male|Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
| 28|Female| Latia Vanhoose|    DB|122502|   27|Latia Vanhoose_Mi...|
| 29|Female| Latia Vanhoose|    DB|152159|   27|Claude Panos_Sant...|
+---+------+---------------+------+------+-----+--------------------+
only showing top 5 rows

+---+------+---------------+------+------+-----+--------------------+
|age|gender|           name|course|  roll|marks|               email|
+---+------+---------------+------+------+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi

8.2 Condición multiple

In [19]:
df.filter(
            (df.course == "DB") & (df.marks > 50)
        ).show(5)

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|  Male| Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
| 28|Female|Mickey Cortright|    DB|192537|   62|Ernest Rossbach_M...|
| 28|Female|     Anna Santos|    DB|311589|   79|Celeste Lollis_Mi...|
| 29|  Male|    Paris Hutton|    DB|481229|   57|Clementina Menke_...|
+---+------+----------------+------+------+-----+--------------------+
only showing top 5 rows



8.3 Funcionalidad __isin()__ para condiciones múltiples

In [24]:
from pyspark.sql.functions import col

l_courses = ["DB", "Cloud", "OOP"]
df.filter(col("course").isin(l_courses)).show(2)
df.filter(df.course.isin(l_courses)).show(2)


+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows



8.4 Condición __startsWith()_

La usamos para determinar cuando una cadena comienza con una determinada letra co caracter

In [29]:
from pyspark.sql.functions import col

df.filter(df.course.startswith("D")).show(2)
df.filter(col("course").startswith("D")).show(2)

+---+------+---------------+------+-----+-----+--------------------+
|age|gender|           name|course| roll|marks|               email|
+---+------+---------------+------+-----+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+---+------+---------------+------+-----+-----+--------------------+
only showing top 2 rows

+---+------+---------------+------+-----+-----+--------------------+
|age|gender|           name|course| roll|marks|               email|
+---+------+---------------+------+-----+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+---+------+---------------+------+-----+-----+--------------------+
only showing top 2 rows



8.5 Filtro con __.contains()__

In [34]:
df.filter(df.name.contains("Eleno")).show(2)
df.filter(col("name").contains("Eleno")).show(2)

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows



8.6 Filtro con __.like()__

In [36]:
from pyspark.sql.functions import col

df.filter(df.name.like("%Ele%")).show(2)
df.filter(col("name").like("%Ele%")).show(2)

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows



9. Spark Ejercicio rápido

In [46]:
from pyspark.sql.functions import col, lit
from pyspark.sql .types import StructField, StructType, IntegerType, StringType

In [139]:
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("emails", StringType(), True),
])

In [140]:
df = spark.read.options(header=True, schema=schema).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
df.show(2)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows



1. Crear una nueva columna __total marks__ con el valor 120


In [49]:
df = df.withColumn("totalMarks", lit(120))

2. Crear una columna __total mark avg__ para cada alumno con la formula 
__(marks/total_marks)*100__

In [50]:
df = df.withColumn("avg", col("marks")/col("totalMarks") * 100)

3. Filtrar los estudiantes con AVG > 80% in __OOP__
4. Filtrar los estudiantes con AVG > 60% in __CLOUD__


In [72]:
df_oop = df.filter(
                    (df.avg>=80) & (df.course == "OOP") 
                )
                 
df_cloud =  df.filter(               
                    (col("avg")>= 60) & (col("course")=="Cloud")
                    )
                


5. Mostrar los nombres y notas de __3__ y __4__


In [77]:
df_oop.select(col("name"), col("marks")).show(2)
df_cloud.select(col("name"), col("marks")).show(2)

+--------------+-----+
|          name|marks|
+--------------+-----+
| Kizzy Brenner|   96|
|Jenna Montague|   98|
+--------------+-----+
only showing top 2 rows

+------------+-----+
|        name|marks|
+------------+-----+
|Claude Panos|   85|
| Billi Clore|   76|
+------------+-----+
only showing top 2 rows



10. Funciones SQL

10.1 Distinct y count

In [83]:
df.select("gender", "age").distinct().show()

+------+---+
|gender|age|
+------+---+
|  Male| 29|
|  Male| 28|
|Female| 28|
|Female| 29|
+------+---+



10.2 dropDuplicates()

In [98]:
df.dropDuplicates(["gender","age"]).show()
df.dropDuplicates(["age"]).show()

+---+------+----------------+------+-----+-----+--------------------+----------+------------------+
|age|gender|            name|course| roll|marks|               email|totalMarks|               avg|
+---+------+----------------+------+-----+-----+--------------------+----------+------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|       120|49.166666666666664|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|       120| 51.66666666666667|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|       120|              37.5|
| 29|  Male|  Cordie Harnois|   OOP|92882|   51|Judie Chipps_Clem...|       120|              42.5|
+---+------+----------------+------+-----+-----+--------------------+----------+------------------+

+---+------+----------------+------+-----+-----+--------------------+----------+------------------+
|age|gender|            name|course| roll|marks|               email|totalMarks|               avg|

11. Spark Ejercicio rápido

In [152]:
from pyspark.sql import SparkSession

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("emails", StringType(), True),
])

df = spark.read.options(schema=schema, header=True).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: string (nullable = true)
 |-- email: string (nullable = true)



Solucion 1.

In [107]:
df2 = df.select("age", "gender", "course").distinct()
df2.count()

24

Solucion 2.

In [109]:
df.dropDuplicates(["gender", "age", "course"]).count()

24

12. .sort() u .orderBy()

In [119]:
df.sort(col('age').asc(), col('marks').desc()).show(4)

+---+------+--------------+------+-------+-----+--------------------+
|age|gender|          name|course|   roll|marks|               email|
+---+------+--------------+------+-------+-----+--------------------+
| 28|Female|Melani Engberg| Cloud|1872667|   99|Alberta Freund_Ni...|
| 28|  Male|   Niki Klimek|   DSA|5172507|   99|Marylee Capasso_E...|
| 28|Female|  Judie Chipps|   OOP|5451977|   99|Tamera Blakley_Mi...|
| 28|Female|Jalisa Swenson|   MVC|5712033|   99|Dustin Feagins_Hu...|
+---+------+--------------+------+-------+-----+--------------------+
only showing top 4 rows



13. Spark ejercicio rapido - Sort() u orderBy()

In [121]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql.functions import col

#employee_name,department,state,salary,age,bonus

schema = StructType([
    StructField('employee_name', StringType(), True),
    StructField('department', StringType(), True),
    StructField('state', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('age', IntegerType(), True),
    StructField('bonus', IntegerType(), True),
])

df_emp = spark.read.options(schema=schema, header = True).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/OfficeData.csv")
df_emp.count()

9

1. Crear un DF ordernado por la columna __bonus__ de forma ascendente


In [124]:
df_emp.select("*").sort(col("bonus").asc()).show(3)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
+-------------+----------+-----+------+---+-----+
only showing top 3 rows



2. Crear un DF ordenado por las columnas __age__ y __salary__ en orden descendente y ascendente.


In [127]:
df_emp.select("*").sort(col("age").desc(), col("salary").asc()).show(3)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|      Michael|     Sales|   NY| 86000| 56|20000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+
only showing top 3 rows



3. Crear un DF ordenado por __age__ __bonus__ y __salario__ en orden descendente, descentente y ascendente.


In [130]:
df_emp.select("*").sort(col("age").desc(), col("bonus").desc(), col("salary").asc()).show(3)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|      Michael|     Sales|   NY| 86000| 56|20000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+
only showing top 3 rows



14. GroupBy en DF

In [12]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


schemas = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("emails", StringType(), True),
])

df = spark.read.schema(schemas).option("header","True").csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")

df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- emails: string (nullable = true)



In [16]:
df.groupBy("gender").count().show()
df.groupBy("gender").sum("marks").show()

+------+-----+
|gender|count|
+------+-----+
|Female|  501|
|  Male|  499|
+------+-----+

+------+----------+
|gender|sum(marks)|
+------+----------+
|Female|     29636|
|  Male|     30461|
+------+----------+

